In [112]:
import pandas as pd
import glob
import os
import re

In [191]:
line_re =re.compile(r'(\d{4})')
cols =['CID10',1,2,3,4,5,6,7,8,9,10,11,12]
dfresult = pd.DataFrame(columns=['CID10', 'MES', 'VALOR', 'ANO','DATA'])

In [192]:
xlsxfiles = [os.path.basename(x) for x in glob.glob('data\\*Previdenciarios.xlsx')]
for cNome in xlsxfiles:
    df = pd.read_excel("data\\"+cNome)
    df = df.rename(columns={'JANEIRO': 1,'FEVEREIRO': 2,'MARÇO': 3,'ABRIL': 4,'MAIO': 5,'JUNHO': 6,'JULHO': 7,'AGOSTO': 8,'SETEMBRO': 9,'OUTUBRO': 10,'NOVEMBRO': 11,'DEZEMBRO': 12 })
    df = df[cols].set_index('CID10').stack().reset_index()
    df = df.rename(columns={'level_1': 'MES', 0: 'VALOR'})
    comp =line_re.search(cNome)
    df['ANO'] =int(comp.group(0))
    df['DATA'] = pd.to_datetime(df.ANO*10000+df.MES*100+1,format='%Y%m%d')
    dfresult = dfresult.append(df)
#a partir de 2017 o formarto no excel é diferente. Foi necessário fazer novo tratamento.
xlsxfiles = [os.path.basename(x) for x in glob.glob('data\\*PrevidenciariosN.xlsx')]
for cNome in xlsxfiles:
    df = pd.read_excel("data\\"+cNome, header=14)
    df['CID10'] = df['CID10 CATEGORIA'].str.extract(r'(\w{1}\d{2})')
    df.dropna(inplace=True)
    df = df[df.CID10 != '999']
    df = df[df.CID10 != '990']
    df=df.drop(['CID10 CATEGORIA'], axis=1)
    df.columns = map(str.upper, df.columns)
    df = df.rename(columns={'JANEIRO': 1,'FEVEREIRO': 2,'MARÇO': 3,'ABRIL': 4,'MAIO': 5,'JUNHO': 6,'JULHO': 7,'AGOSTO': 8,'SETEMBRO': 9,'OUTUBRO': 10,'NOVEMBRO': 11,'DEZEMBRO': 12,'JUNÃO':6 })
    df = df[cols].set_index('CID10').stack().reset_index()
    df = df.rename(columns={'level_1': 'MES', 0: 'VALOR'})
    comp =line_re.search(cNome)
    df['ANO'] =int(comp.group(0)) 
    df['DATA'] = pd.to_datetime(df.ANO*10000+df.MES*100+1,format='%Y%m%d')
    dfresult = dfresult.append(df)
dfresult['CID10'] = dfresult['CID10'].fillna('–')
dfresult.to_csv('data\\agrupado.csv')
dfresult.to_excel('data\\agrupado.xlsx')